# Team 3 - Final Project Data Processing

In [2]:
!pip install pyathena

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [78]:
import boto3
from botocore.exceptions import ClientError
import sagemaker
import pyathena as pa
from pyathena.pandas.cursor import PandasCursor
from datetime import datetime
import os

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [5]:
# Athena modeling bucket
s3_modeling_dir = 's3://ads508team3/modeling/'
# Athena staging bucket
s3_staging_dir = 's3://ads508team3/athena-staging/'

In [6]:
# Create Connection
conn = pa.connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [7]:
dbname = 'myanimelist'
table_list = ['animelist', 'anime', 'synopsis', 'ratings', 'watching_status']

### Construct DataFrames

**Bypassing the default pyathena cursor and using a more efficient PandasCursor**

In [8]:
cursor = pa.connect(s3_staging_dir=s3_staging_dir, region_name='us-east-1',cursor_class=PandasCursor).cursor()

In [9]:
anime_list = cursor.execute('SELECT * FROM %s.%s' % (dbname, 'animelist')).as_pandas()
anime = cursor.execute('SELECT * FROM %s.%s' % (dbname, 'anime')).as_pandas()
#synopsis = cursor.execute('SELECT * FROM %s.%s' % (dbname, 'synopsis')).as_pandas()
#ratings = cursor.execute('SELECT * FROM %s.%s' % (dbname, 'ratings')).as_pandas()
watching_status = cursor.execute('SELECT * FROM %s.%s' % (dbname, 'watching_status')).as_pandas()

# Data Preparation

### Fix anime scores, replace 'Unknown' values with np.nan and then mean impute

In [10]:
anime.dtypes

mal_id            Int64
name             object
score            object
genres           object
english_name     object
japanese_name    object
type             object
episodes         object
aired            object
premiered        object
producers        object
licensors        object
studios          object
source           object
duration         object
rating           object
ranked           object
popularity        Int64
members           Int64
favorites         Int64
watching          Int64
completed         Int64
on_hold           Int64
dropped           Int64
plan_to_watch    object
score_10         object
score_9          object
score_8          object
score_7          object
score_6          object
score_5          object
score_4          object
score_3          object
score_2          object
score_1          object
dtype: object

In [12]:
# Replace Unknown Values
anime['score'] = anime['score'].replace({'Unknown': np.nan})
anime['score'] = anime['score'].astype('float')

In [15]:
anime['score'].dtypes

dtype('float64')

In [16]:
anime['score'].isna().sum()

5141

In [17]:
# We may have to change this imputation method if it impacts the model too much
anime['score'] = anime['score'].fillna(anime['score'].mean())

In [18]:
anime['score']

0        6.120000
1        6.970000
2        5.960000
3        5.740000
4        6.010000
           ...   
17557    6.509999
17558    6.509999
17559    6.509999
17560    6.509999
17561    6.509999
Name: score, Length: 17562, dtype: float64

### Feature Days Since Aired
Pull the first date from Aired, convert to datetime and get timedelta in days

In [19]:
anime['days_since_aired'] = anime['aired'].str.split(' to ', expand=True).iloc[:, 0]

In [20]:
# Remove spaces left and right
anime['days_since_aired'] = anime['days_since_aired'].str.strip()

In [21]:
# Fix date format issues
anime['days_since_aired'] = anime['days_since_aired'].str.replace('^\w{3}, \d{4}', lambda x: x[0][0:3] + ' 1, ' + x[0][5:9], regex=True)
anime['days_since_aired'] = anime['days_since_aired'].str.replace('^\d{4}', lambda x: 'Jan 1, ' + x[0], regex=True)
anime['days_since_aired']

0        Jan 11, 2002
1        Jul 25, 1995
2         Apr 7, 1999
3        Dec 21, 1989
4        Aug 21, 1999
             ...     
17557     Apr 4, 2021
17558     Jan 1, 2021
17559     Jul 1, 2021
17560         Unknown
17561     Jul 1, 2021
Name: days_since_aired, Length: 17562, dtype: object

In [22]:
anime['days_since_aired'] = anime['days_since_aired'].replace('Unknown', np.nan)
anime['days_since_aired']

0        Jan 11, 2002
1        Jul 25, 1995
2         Apr 7, 1999
3        Dec 21, 1989
4        Aug 21, 1999
             ...     
17557     Apr 4, 2021
17558     Jan 1, 2021
17559     Jul 1, 2021
17560             NaN
17561     Jul 1, 2021
Name: days_since_aired, Length: 17562, dtype: object

In [23]:
anime['days_since_aired'] = pd.to_datetime(anime['days_since_aired'], format='%b %d, %Y')

In [24]:
anime['days_since_aired'] = anime['days_since_aired'].fillna(pd.datetime.now())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [25]:
anime['days_since_aired']

0       2002-01-11 00:00:00.000000
1       1995-07-25 00:00:00.000000
2       1999-04-07 00:00:00.000000
3       1989-12-21 00:00:00.000000
4       1999-08-21 00:00:00.000000
                   ...            
17557   2021-04-04 00:00:00.000000
17558   2021-01-01 00:00:00.000000
17559   2021-07-01 00:00:00.000000
17560   2022-03-31 18:19:05.994792
17561   2021-07-01 00:00:00.000000
Name: days_since_aired, Length: 17562, dtype: datetime64[ns]

### Get Time Delta in days

In [26]:
anime['days_since_aired'] = (pd.datetime.now() - anime['days_since_aired']).dt.days

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [27]:
anime['days_since_aired']

0         7384
1         9746
2         8394
3        11788
4         8258
         ...  
17557      361
17558      454
17559      273
17560        0
17561      273
Name: days_since_aired, Length: 17562, dtype: int64

# One Hot Encode Genres

In [29]:
genre_list = anime['genres'].apply(lambda x: x.split(', '))

In [30]:
# Unique Genre
list(set(genre_list.sum()))

['Thriller',
 'Demons',
 'Comedy',
 'Mystery',
 'Drama',
 'Adventure',
 'Seinen',
 'Game',
 'School',
 'Josei',
 'Action',
 'Horror',
 'Ecchi',
 'Romance',
 'Harem',
 'Space',
 'Unknown',
 'Music',
 'Mecha',
 'Supernatural',
 'Military',
 'Slice of Life',
 'Psychological',
 'Martial Arts',
 'Sci-Fi',
 'Hentai',
 'Parody',
 'Yaoi',
 'Historical',
 'Super Power',
 'Shoujo',
 'Samurai',
 'Kids',
 'Shounen',
 'Shoujo Ai',
 'Police',
 'Dementia',
 'Cars',
 'Vampire',
 'Fantasy',
 'Sports',
 'Shounen Ai',
 'Yuri',
 'Magic']

In [31]:
mlb = MultiLabelBinarizer()
ohe = pd.DataFrame(mlb.fit_transform(genre_list), columns=mlb.classes_, index=genre_list.index)

In [32]:
anime = pd.concat([anime, ohe], axis=1)

In [33]:
anime

,mal_id,name,score,genres,english_name,japanese_name,type,episodes,aired,premiered,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,1224,Aquarian Age: Sign for Evolution,6.120000,"Action, Sci-Fi, Adventure, Music, Super Power,...",Aquarian Age:Sign for Evolution,アクエリアンエイジ Sign for Evolution,TV,13,"Jan 11, 2002 to Mar 29, 2002",Winter 2002,...,0,0,0,1,1,0,0,0,0,0
1,1225,3x3 Eyes: Seima Densetsu,6.970000,"Action, Adventure, Horror, Demons, Romance, Fa...",3x3 Eyes:Legend of the Divine Demon,3×3 EYES聖魔伝説,OVA,3,"Jul 25, 1995 to Jun 25, 1996",Unknown,...,0,0,0,0,0,0,0,0,0,0
2,1226,Seihou Tenshi Angel Links,5.960000,"Action, Adventure, Space, Comedy, Romance, Dra...",Angel Links,星方天使エンジェルリンクス,TV,13,"Apr 7, 1999 to Jun 30, 1999",Spring 1999,...,0,1,0,0,0,0,0,0,0,0
3,1227,Shuten Douji,5.740000,"Space, Supernatural, Demons, Historical, Horro...",Shuten Doji - The Star Hand Kid,手天童子,OVA,4,"Dec 21, 1989 to Dec 21, 1991",Unknown,...,0,1,0,0,1,0,0,0,0,0
4,1228,Chivas 1-2-3,6.010000,"Action, Comedy, Ecchi, Fantasy, Supernatural",Sorcerer on the Rocks,シーバス１-２-３,OVA,2,"Aug 21, 1999 to Sep 21, 1999",Unknown,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,6.509999,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,...,0,0,0,0,1,0,0,0,0,0
17558,48483,Mieruko-chan,6.509999,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,...,0,0,0,0,1,0,0,0,0,0
17559,48488,Higurashi no Naku Koro ni Sotsu,6.509999,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,Unknown,"Jul, 2021 to ?",Summer 2021,...,0,0,0,0,1,1,0,0,0,0
17560,48491,Yama no Susume: Next Summit,6.509999,"Adventure, Slice of Life, Comedy",Unknown,ヤマノススメ Next Summit,TV,Unknown,Unknown,Unknown,...,1,0,0,0,0,0,0,0,0,0


### One hot encode Type

In [35]:
anime['type'].unique()

array(['TV', 'OVA', 'Special', 'Movie', 'ONA', 'Music', 'Unknown'],
      dtype=object)

In [36]:
anime['type'] = anime['type'].replace({'Unknown': 'Other'})

In [38]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
type_ohe = pd.DataFrame(enc.fit_transform(anime[['type']]), columns=list(enc.categories_[0]))

In [39]:
type_ohe

,Movie,Music,ONA,OVA,Other,Special,TV
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
17557,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17558,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17559,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17560,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
# Concat
anime = pd.concat([anime, type_ohe], axis=1)

In [44]:
anime

,mal_id,name,score,genres,english_name,japanese_name,type,episodes,aired,premiered,...,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Other,Special,TV
0,1224,Aquarian Age: Sign for Evolution,6.120000,"Action, Sci-Fi, Adventure, Music, Super Power,...",Aquarian Age:Sign for Evolution,アクエリアンエイジ Sign for Evolution,TV,13,"Jan 11, 2002 to Mar 29, 2002",Winter 2002,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1225,3x3 Eyes: Seima Densetsu,6.970000,"Action, Adventure, Horror, Demons, Romance, Fa...",3x3 Eyes:Legend of the Divine Demon,3×3 EYES聖魔伝説,OVA,3,"Jul 25, 1995 to Jun 25, 1996",Unknown,...,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1226,Seihou Tenshi Angel Links,5.960000,"Action, Adventure, Space, Comedy, Romance, Dra...",Angel Links,星方天使エンジェルリンクス,TV,13,"Apr 7, 1999 to Jun 30, 1999",Spring 1999,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1227,Shuten Douji,5.740000,"Space, Supernatural, Demons, Historical, Horro...",Shuten Doji - The Star Hand Kid,手天童子,OVA,4,"Dec 21, 1989 to Dec 21, 1991",Unknown,...,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1228,Chivas 1-2-3,6.010000,"Action, Comedy, Ecchi, Fantasy, Supernatural",Sorcerer on the Rocks,シーバス１-２-３,OVA,2,"Aug 21, 1999 to Sep 21, 1999",Unknown,...,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,Daomu Biji Zhi Qinling Shen Shu,6.509999,"Adventure, Mystery, Supernatural",Unknown,盗墓笔记之秦岭神树,ONA,Unknown,"Apr 4, 2021 to ?",Unknown,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
17558,48483,Mieruko-chan,6.509999,"Comedy, Horror, Supernatural",Unknown,見える子ちゃん,TV,Unknown,2021 to ?,Unknown,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17559,48488,Higurashi no Naku Koro ni Sotsu,6.509999,"Mystery, Dementia, Horror, Psychological, Supe...",Higurashi:When They Cry – SOTSU,ひぐらしのなく頃に卒,TV,Unknown,"Jul, 2021 to ?",Summer 2021,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17560,48491,Yama no Susume: Next Summit,6.509999,"Adventure, Slice of Life, Comedy",Unknown,ヤマノススメ Next Summit,TV,Unknown,Unknown,Unknown,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Rating Ordinal Encoding

Note that Unknown variables are also grouped along with the most severe rating, just in case. Don't want to accidentally recommend a Adult title if one of the Unknown rating is an adult title. It would be bad to recommend an adult title to the wrong audience

In [46]:
anime['rating'].unique()

array(['PG-13 - Teens 13 or older', 'R - 17+ (violence & profanity)',
       'R+ - Mild Nudity', 'G - All Ages', 'Rx - Hentai', 'PG - Children',
       'Unknown'], dtype=object)

In [47]:
# Order G to Adult from 1-6
anime['rating'] = anime['rating'].replace({'G - All Ages': 1, 'PG - Children': 2, 
                                           'PG-13 - Teens 13 or older': 3, 'R+ - Mild Nudity': 4,
                                           'R - 17+ (violence & profanity)': 5, 'Rx - Hentai': 6,
                                           'Unknown': 6})

In [48]:
anime['rating']

0        3
1        5
2        3
3        4
4        5
        ..
17557    6
17558    6
17559    5
17560    3
17561    6
Name: rating, Length: 17562, dtype: int64

## Merging dataframes

In [50]:
# First subset the anime table
anime_cols = ['mal_id', 'score', 'episodes', 'rating', 'ranked', 'popularity',
       'members', 'score_10', 'score_9', 'score_8', 'score_7', 'score_6',
       'score_5', 'score_4', 'score_3', 'score_2', 'score_1',
       'days_since_aired', 'Action', 'Adventure', 'Cars', 'Comedy', 'Dementia',
       'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem', 'Hentai',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'Martial Arts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai', 'Slice of Life',
       'Space', 'Sports', 'Super Power', 'Supernatural', 'Thriller', 'Unknown',
       'Vampire', 'Yaoi', 'Yuri', 'Movie', 'Music', 'ONA', 'OVA', 'Other',
       'Special', 'TV']

anime_subset = anime[anime_cols]

In [52]:
# Global replace 'Unknown'
anime_subset = anime_subset.replace({'Unknown': np.nan})

### Imputation

In [53]:
anime_subset

,mal_id,score,episodes,rating,ranked,popularity,members,score_10,score_9,score_8,...,Yaoi,Yuri,Movie,Music,Music,ONA,OVA,Other,Special,TV
0,1224,6.120000,13,3,7679.0,4575,11838,104.0,197.0,454.0,...,0,0,0.0,1,0.0,0.0,0.0,0.0,0.0,1.0
1,1225,6.970000,3,5,3730.0,4491,12438,357.0,624.0,1371.0,...,0,0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0
2,1226,5.960000,13,3,8369.0,5534,7429,69.0,90.0,202.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,1227,5.740000,4,4,9036.0,8310,2488,21.0,21.0,72.0,...,0,0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0
4,1228,6.010000,2,5,8180.0,7802,3024,47.0,47.0,114.0,...,0,0,0.0,0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,48481,6.509999,NaN,6,NaN,13116,354,NaN,NaN,NaN,...,0,0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0
17558,48483,6.509999,NaN,6,NaN,17562,7010,NaN,NaN,NaN,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
17559,48488,6.509999,NaN,5,NaN,17558,11309,1.0,NaN,NaN,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
17560,48491,6.509999,NaN,3,NaN,17565,1386,NaN,NaN,NaN,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [55]:
anime_subset['episodes'] = anime_subset['episodes'].fillna(anime_subset['episodes'].median())
anime_subset['ranked'] = anime_subset['ranked'].fillna(anime_subset['ranked'].median())

for s in range(1, 11):
    anime_subset['score_' + str(s)] = anime_subset['score_' + str(s)].fillna(anime_subset['score_' + str(s)].median())

In [57]:
anime_subset.isna().sum()

mal_id      0
score       0
episodes    0
rating      0
ranked      0
           ..
ONA         0
OVA         0
Other       0
Special     0
TV          0
Length: 71, dtype: int64

In [58]:
# Rename rating in anime_list to prevent confusion
anime_list.rename(columns={'rating': 'user_rating'}, inplace=True)

In [59]:
anime_list = pd.merge(anime_list, watching_status, left_on='watching_status', right_on='status')

In [60]:
anime_list

,user_id,anime_id,user_rating,watching_status,watched_episodes,status,description
0,336145,2251,10,2,13,2,Completed
1,343247,2787,9,2,24,2,Completed
2,196188,22729,8,2,12,2,Completed
3,161132,1119,7,2,1,2,Completed
4,175224,20909,6,2,7,2,Completed
...,...,...,...,...,...,...,...
999991,315329,16,6,4,9,4,Dropped
999992,20341,1887,0,4,3,4,Dropped
999993,5720,18679,4,4,1,4,Dropped
999994,291821,4898,0,4,0,4,Dropped


In [62]:
anime_list = anime_list.drop(columns=['status'])
anime_list

,user_id,anime_id,user_rating,watching_status,watched_episodes,description
0,336145,2251,10,2,13,Completed
1,343247,2787,9,2,24,Completed
2,196188,22729,8,2,12,Completed
3,161132,1119,7,2,1,Completed
4,175224,20909,6,2,7,Completed
...,...,...,...,...,...,...
999991,315329,16,6,4,9,Dropped
999992,20341,1887,0,4,3,Dropped
999993,5720,18679,4,4,1,Dropped
999994,291821,4898,0,4,0,Dropped


In [64]:
processed_df = pd.merge(anime_list, anime_subset, left_on='anime_id', right_on='mal_id')

In [66]:
processed_df.drop(columns=['description'], inplace=True)
# Rename Score to aggregated score
processed_df.rename(columns={'score': 'aggregated_score'}, inplace=True)

In [67]:
processed_df

,user_id,anime_id,user_rating,watching_status,watched_episodes,mal_id,aggregated_score,episodes,rating,ranked,...,Yaoi,Yuri,Movie,Music,Music,ONA,OVA,Other,Special,TV
0,336145,2251,10,2,13,2251,8.420000,13,5,143.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,228773,2251,9,2,13,2251,8.420000,13,5,143.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
2,61090,2251,10,2,13,2251,8.420000,13,5,143.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,26848,2251,9,2,13,2251,8.420000,13,5,143.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,195185,2251,8,2,13,2251,8.420000,13,5,143.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999991,300428,13503,0,4,0,13503,6.509999,11,3,13708.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
999992,30459,34369,0,4,0,34369,6.509999,5,3,14403.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0
999993,275530,7779,0,4,0,7779,6.210000,52,2,7295.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0
999994,272555,40406,0,4,0,40406,6.509999,16,1,15754.0,...,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,1.0


# Save Merged and Cleaned Dataset to S3

In [76]:
# Save as csv file to S3
processed_df.to_csv('Data/training.csv', index=False)

In [79]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [82]:
upload_file('Data/training.csv', 'ads508team3', 'modeling/training.csv')

True